# Book Recommendation System

In [1]:
import numpy as np
import pandas as pd

In [2]:
# read books dataset
book_df = pd.read_csv("books_clean.csv")
display(book_df.head())

,id,etag,selfLink,volumeInfo.title,volumeInfo.subtitle,volumeInfo.authors,volumeInfo.publisher,volumeInfo.description,volumeInfo.readingModes.text,volumeInfo.readingModes.image,...,accessInfo.pdf.isAvailable,accessInfo.accessViewStatus,searchInfo.textSnippet,volumeInfo.averageRating,volumeInfo.ratingsCount,volumeInfo.publishedYear,volumeInfo.industryIdentifiers.ISBN_10,volumeInfo.industryIdentifiers.ISBN_13,volumeInfo.industryIdentifiers.ISSN,volumeInfo.industryIdentifiers.OTHER
0,eIJiDwAAQBAJ,1xcHQ4mb0QM,https://www.googleapis.com/books/v1/volumes/eI...,Invest Outside the Box,Understanding Different Asset Classes and Stra...,Tariq Dennison,Springer,This book is a practical and concise guide to ...,True,True,...,True,SAMPLE,This book is a practical and concise guide to ...,0.0,0,2018,981130372X,9.78981E+12,Missing,Missing
1,UacEEAAAQBAJ,pQzVcEsJ4kI,https://www.googleapis.com/books/v1/volumes/Ua...,How I Invest My Money,Finance experts reveal how they save spend and...,"Brian Portnoy, Joshua Brown",Harriman House Limited,The world of investing normally sees experts t...,True,True,...,True,SAMPLE,Sometimes raw always revealing these stories d...,0.0,0,2020,857198092,9.78086E+12,Missing,Missing
2,pNI4DwAAQBAJ,2V7L2ChAUb4,https://www.googleapis.com/books/v1/volumes/pN...,Value Investing in Asia,The Definitive Guide to Investing in Asia,"Peir Shenq Stanley Lim, Mun Hong Cheong",John Wiley & Sons,A practical stepbystep guide to value investin...,True,True,...,True,SAMPLE,Ltd Stanley Lim and Mun Hong have skillfully w...,0.0,0,2017,1119391202,9.78112E+12,Missing,Missing
3,DBeCEAAAQBAJ,GzHy/Dvmvl0,https://www.googleapis.com/books/v1/volumes/DB...,How to Invest,Masters on the Craft,David M Rubenstein,Simon and Schuster,What do the most successful investors have in ...,False,False,...,False,SAMPLE,David M Rubenstein cofounder of one of the wor...,0.0,0,2022,1982190302,9.78198E+12,Missing,Missing
4,5qCaDwAAQBAJ,7xVJ4bf3Hfc,https://www.googleapis.com/books/v1/volumes/5q...,Simply Invest,Naked Truths to Grow Your Money,Goh Yang Chye,Epigram Books,A pioneer in financial planning in Singapore G...,True,False,...,False,SAMPLE,From understanding market cycles to knowing wh...,0.0,0,2019,981478561X,9.78981E+12,Missing,Missing


## Text Processing

In [3]:
# keep only necessary columns
book_rec_df = book_df[['volumeInfo.title', 'volumeInfo.authors', 'volumeInfo.description', 'volumeInfo.categories']]

# replace Missing with empty space
book_rec_df = book_rec_df.replace("Missing", '')

# add a column： genre + author + title + description
book_rec_df['text'] = book_rec_df[['volumeInfo.title', 'volumeInfo.authors', 'volumeInfo.description', 'volumeInfo.categories']].apply(" ".join, axis=1)

display(book_rec_df.head())

,volumeInfo.title,volumeInfo.authors,volumeInfo.description,volumeInfo.categories,text
0,Invest Outside the Box,Tariq Dennison,This book is a practical and concise guide to ...,Business & Economics,Invest Outside the Box Tariq Dennison This boo...
1,How I Invest My Money,"Brian Portnoy, Joshua Brown",The world of investing normally sees experts t...,Business & Economics,"How I Invest My Money Brian Portnoy, Joshua Br..."
2,Value Investing in Asia,"Peir Shenq Stanley Lim, Mun Hong Cheong",A practical stepbystep guide to value investin...,Business & Economics,Value Investing in Asia Peir Shenq Stanley Lim...
3,How to Invest,David M Rubenstein,What do the most successful investors have in ...,Business & Economics,How to Invest David M Rubenstein What do the m...
4,Simply Invest,Goh Yang Chye,A pioneer in financial planning in Singapore G...,Business & Economics,Simply Invest Goh Yang Chye A pioneer in finan...


In [4]:
# Random Checking
pd.set_option('display.max_colwidth', None)
book_rec_df.loc[book_rec_df['volumeInfo.title']=='Invest Outside the Box', ['text']]

,text
0,Invest Outside the Box Tariq Dennison This book is a practical and concise guide to major asset classes investment strategies and foreign markets For investors familiar with one box of investments this book serves as a nontechnical introduction to other boxes worth diversifying into such as bonds real estate private equity cryptocurrencies and Chinese Ashares Readers with no prior finance background will find this book an accessible entry point to investing Written by a practitioner this volume can serve as course material for introductory investing classes or as an onthejob guidebook for professionals and practicing investors Business & Economics


In [22]:
# Text Pre-Processing
import nltk
import string
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer

# Function for removing punctuations
def remove_punctuation(text):
    text = "".join([i for i in text if i not in string.punctuation])
    return text

# Function for converting into lower case
def lower_case(text):
    return text.lower()

# Function for tokenization
def tokenization(text):
    tokens = nltk.word_tokenize(text)
    return tokens

# Function for removing stop words
def remove_stop_words(text):
    stopwords = nltk.corpus.stopwords.words('english')
    text = [i for i in text if i not in stopwords]
    return text

# Function for stemming
porter_stemmer = PorterStemmer()
def stem_text(text):
    stem_text = [porter_stemmer.stem(i) for i in text]
    return stem_text

# Function for lemmatization
wordnet_lemmatizer = WordNetLemmatizer()
def lemmatize_text(text):
    lemm_text = [wordnet_lemmatizer.lemmatize(i) for i in text]
    return lemm_text

In [23]:
# Perform text preprocessing
book_rec_df['text_no_punc'] = book_rec_df['text'].apply(lambda x:remove_punctuation(x))
book_rec_df['text_lower'] = book_rec_df['text_no_punc'].apply(lambda x:lower_case(x))
book_rec_df['text_token'] = book_rec_df['text_lower'].apply(lambda x:tokenization(x))
book_rec_df['text_no_stop'] = book_rec_df['text_token'].apply(lambda x:remove_stop_words(x))
book_rec_df['text_stem'] = book_rec_df['text_no_stop'].apply(lambda x:stem_text(x))
book_rec_df['text_lemm'] = book_rec_df['text_stem'].apply(lambda x:lemmatize_text(x))

In [25]:
# Random Checking
pd.set_option('display.max_colwidth', None)
book_rec_df.loc[book_rec_df['volumeInfo.title']=='Invest Outside the Box', ['text', 'text_lemm']]

,text,text_lemm
0,Invest Outside the Box Tariq Dennison This book is a practical and concise guide to major asset classes investment strategies and foreign markets For investors familiar with one box of investments this book serves as a nontechnical introduction to other boxes worth diversifying into such as bonds real estate private equity cryptocurrencies and Chinese Ashares Readers with no prior finance background will find this book an accessible entry point to investing Written by a practitioner this volume can serve as course material for introductory investing classes or as an onthejob guidebook for professionals and practicing investors Business & Economics,"[invest, outsid, box, tariq, dennison, book, practic, concis, guid, major, asset, class, invest, strategi, foreign, market, investor, familiar, one, box, invest, book, serv, nontechn, introduct, box, worth, diversifi, bond, real, estat, privat, equiti, cryptocurr, chine, ashar, reader, prior, financ, background, find, book, access, entri, point, invest, written, practition, volum, serv, cours, materi, introductori, invest, class, onthejob, guidebook, profession, practic, investor, busi, econom]"


In [26]:
# Text preprocessed
book_rec_df['text_preprocessed'] = book_rec_df['text_lemm']

# Combine into sentence
book_rec_df['sentence_preprocessed'] = book_rec_df['text_preprocessed'].apply(lambda x:" ".join(x))

In [27]:
display(book_rec_df.head())

,volumeInfo.title,volumeInfo.authors,volumeInfo.description,volumeInfo.categories,text,text_preprocessed,text_no_punc,text_lower,text_token,text_no_stop,text_stem,text_lemm,sentence_preprocessed
0,Invest Outside the Box,Tariq Dennison,This book is a practical and concise guide to major asset classes investment strategies and foreign markets For investors familiar with one box of investments this book serves as a nontechnical introduction to other boxes worth diversifying into such as bonds real estate private equity cryptocurrencies and Chinese Ashares Readers with no prior finance background will find this book an accessible entry point to investing Written by a practitioner this volume can serve as course material for introductory investing classes or as an onthejob guidebook for professionals and practicing investors,Business & Economics,Invest Outside the Box Tariq Dennison This book is a practical and concise guide to major asset classes investment strategies and foreign markets For investors familiar with one box of investments this book serves as a nontechnical introduction to other boxes worth diversifying into such as bonds real estate private equity cryptocurrencies and Chinese Ashares Readers with no prior finance background will find this book an accessible entry point to investing Written by a practitioner this volume can serve as course material for introductory investing classes or as an onthejob guidebook for professionals and practicing investors Business & Economics,"[invest, outsid, box, tariq, dennison, book, practic, concis, guid, major, asset, class, invest, strategi, foreign, market, investor, familiar, one, box, invest, book, serv, nontechn, introduct, box, worth, diversifi, bond, real, estat, privat, equiti, cryptocurr, chine, ashar, reader, prior, financ, background, find, book, access, entri, point, invest, written, practition, volum, serv, cours, materi, introductori, invest, class, onthejob, guidebook, profession, practic, investor, busi, econom]",Invest Outside the Box Tariq Dennison This book is a practical and concise guide to major asset classes investment strategies and foreign markets For investors familiar with one box of investments this book serves as a nontechnical introduction to other boxes worth diversifying into such as bonds real estate private equity cryptocurrencies and Chinese Ashares Readers with no prior finance background will find this book an accessible entry point to investing Written by a practitioner this volume can serve as course material for introductory investing classes or as an onthejob guidebook for professionals and practicing investors Business Economics,invest outside the box tariq dennison this book is a practical and concise guide to major asset classes investment strategies and foreign markets for investors familiar with one box of investments this book serves as a nontechnical introduction to other boxes worth diversifying into such as bonds real estate private equity cryptocurrencies and chinese ashares readers with no prior finance background will find this book an accessible entry point to investing written by a practitioner this volume can serve as course material for introductory investing classes or as an onthejob guidebook for professionals and practicing investors business economics,"[invest, outside, the, box, tariq, dennison, this, book, is, a, practical, and, concise, guide, to, major, asset, classes, investment, strategies, and, foreign, markets, for, investors, familiar, with, one, box, of, investments, this, book, serves, as, a, nontechnical, introduction, to, other, boxes, worth, diversifying, into, such, as, bonds, real, estate, private, equity, cryptocurrencies, and, chinese, ashares, readers, with, no, prior, finance, background, will, find, this, book, an, accessible, entry, point, to, investing, written, by, a, practitioner, this, volume, can, serve, as, course, material, for, introductory, investing, classes, or, as, an, onthejob, guidebo

## Sentiment Analysis on Preprocessed text

In [28]:
# Sentiment Analysis using Vader
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

analyzer = SentimentIntensityAnalyzer()
book_rec_df['compound_score'] = [analyzer.polarity_scores(x)['compound'] for x in book_rec_df['sentence_preprocessed']]
book_rec_df['negative_score'] = [analyzer.polarity_scores(x)['neg'] for x in book_rec_df['sentence_preprocessed']]
book_rec_df['neutral_score'] = [analyzer.polarity_scores(x)['neu'] for x in book_rec_df['sentence_preprocessed']]
book_rec_df['positive_score'] = [analyzer.polarity_scores(x)['pos'] for x in book_rec_df['sentence_preprocessed']]

In [29]:
display(book_rec_df.head())

,volumeInfo.title,volumeInfo.authors,volumeInfo.description,volumeInfo.categories,text,text_preprocessed,text_no_punc,text_lower,text_token,text_no_stop,text_stem,text_lemm,sentence_preprocessed,compound_score,negative_score,neutral_score,positive_score
0,Invest Outside the Box,Tariq Dennison,This book is a practical and concise guide to major asset classes investment strategies and foreign markets For investors familiar with one box of investments this book serves as a nontechnical introduction to other boxes worth diversifying into such as bonds real estate private equity cryptocurrencies and Chinese Ashares Readers with no prior finance background will find this book an accessible entry point to investing Written by a practitioner this volume can serve as course material for introductory investing classes or as an onthejob guidebook for professionals and practicing investors,Business & Economics,Invest Outside the Box Tariq Dennison This book is a practical and concise guide to major asset classes investment strategies and foreign markets For investors familiar with one box of investments this book serves as a nontechnical introduction to other boxes worth diversifying into such as bonds real estate private equity cryptocurrencies and Chinese Ashares Readers with no prior finance background will find this book an accessible entry point to investing Written by a practitioner this volume can serve as course material for introductory investing classes or as an onthejob guidebook for professionals and practicing investors Business & Economics,"[invest, outsid, box, tariq, dennison, book, practic, concis, guid, major, asset, class, invest, strategi, foreign, market, investor, familiar, one, box, invest, book, serv, nontechn, introduct, box, worth, diversifi, bond, real, estat, privat, equiti, cryptocurr, chine, ashar, reader, prior, financ, background, find, book, access, entri, point, invest, written, practition, volum, serv, cours, materi, introductori, invest, class, onthejob, guidebook, profession, practic, investor, busi, econom]",Invest Outside the Box Tariq Dennison This book is a practical and concise guide to major asset classes investment strategies and foreign markets For investors familiar with one box of investments this book serves as a nontechnical introduction to other boxes worth diversifying into such as bonds real estate private equity cryptocurrencies and Chinese Ashares Readers with no prior finance background will find this book an accessible entry point to investing Written by a practitioner this volume can serve as course material for introductory investing classes or as an onthejob guidebook for professionals and practicing investors Business Economics,invest outside the box tariq dennison this book is a practical and concise guide to major asset classes investment strategies and foreign markets for investors familiar with one box of investments this book serves as a nontechnical introduction to other boxes worth diversifying into such as bonds real estate private equity cryptocurrencies and chinese ashares readers with no prior finance background will find this book an accessible entry point to investing written by a practitioner this volume can serve as course material for introductory investing classes or as an onthejob guidebook for professionals and practicing investors business economics,"[invest, outside, the, box, tariq, dennison, this, book, is, a, practical, and, concise, guide, to, major, asset, classes, investment, strategies, and, foreign, markets, for, investors, familiar, with, one, box, of, investments, this, book, serves, as, a, nontechnical, introduction, to, other, boxes, worth, diversifying, into, such, as, bonds, real, estate, private, equity, cryptocurrencies, and, chinese, ashares, readers, with, no, prior, finance, background, will, find, this, book, an, accessible, entry, point, to, investing, written, by, a, practitioner, this, volume, can, serve, as, course, material, for, intr